In [3]:
import os

In [4]:
os.chdir("../")
%pwd

'c:\\Users\\Maza\\Desktop\\job_aplication_agent'

In [26]:
from dataclasses import dataclass, field
from pathlib import Path
from typing import Union

@dataclass(frozen=True)
class CVConfig:
    CV_file: Union[Path, str]
    job_description: Union[Path, str]
    qualifications: Union[Path, str]
    contact_info: Union[Path, str]
    about_me: Union[Path, str]
    education: Union[Path, str]
    experience: Union[Path, str]
    model: str

    def __post_init__(self):
        # Convert string paths to Path objects
        object.__setattr__(self, 'CV_file', Path(self.CV_file))
        object.__setattr__(self, 'job_description', Path(self.job_description))
        object.__setattr__(self, 'qualifications', Path(self.qualifications))
        object.__setattr__(self, 'contact_info', Path(self.contact_info))
        object.__setattr__(self, 'about_me', Path(self.about_me))
        object.__setattr__(self, 'education', Path(self.education))
        object.__setattr__(self, 'experience', Path(self.experience))


In [27]:
from dotenv import load_dotenv
from pathlib import Path


from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser

from job_aplication_agent.utils.common import  load_json,read_yaml
from job_aplication_agent import logger
from job_aplication_agent.constants import *
load_dotenv()

True

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        models_filepath = MODELS_FILE_PATH,
        prompt_template = PROMPT_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.models = read_yaml(models_filepath)
        self.prompt_template = read_yaml(prompt_template)
    
        

    
    def get_CV_config(self) -> CVConfig:
        """
        
        Returns:
            
        """
        
        CV_config = self.config.CV
        # prompt_template = self.prompt_template
        
        create_CV_config = CVConfig(
            CV_file= CV_config.CV_file,
            job_description= CV_config.job_description,
            qualifications= CV_config.qualifications,
            contact_info=CV_config.contact_info,
            about_me=CV_config.about_me,
            education=CV_config.education,
            experience=CV_config.experience,
            model= self.models.Llama3

           
        ) 

        return create_CV_config

In [29]:
config = ConfigurationManager()
cv_config = config.get_CV_config()


[2024-06-27 15:44:43,842: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-06-27 15:44:43,844: INFO: common: yaml file: models.yaml loaded successfully:]
[2024-06-27 15:44:43,847: INFO: common: yaml file: prompt_template.yaml loaded successfully:]
prompt_text_inputs\job_description.txt


In [31]:
type(cv_config.job_description)

pathlib.WindowsPath

In [32]:
with open(cv_config.CV_file, 'r') as file:
    content = file.readlines()

In [33]:
content

['\\documentclass[letterpaper,10pt]{article}\n',
 '\n',
 '\\usepackage{latexsym}\n',
 '\\usepackage[empty]{fullpage}\n',
 '\\usepackage{titlesec}\n',
 '\\usepackage{marvosym}\n',
 '\\usepackage[usenames,dvipsnames]{color}\n',
 '\\usepackage{verbatim}\n',
 '\\usepackage{enumitem}\n',
 '\\usepackage[hidelinks]{hyperref}\n',
 '\\usepackage{fancyhdr}\n',
 '\\usepackage[english]{babel}\n',
 '\\usepackage{tabularx}\n',
 '\\input{glyphtounicode}\n',
 '\n',
 '% Font options\n',
 '\\usepackage[sfdefault]{roboto}  % Sans-serif font\n',
 '\n',
 '\\pagestyle{fancy}\n',
 '\\fancyhf{}\n',
 '\\fancyfoot{}\n',
 '\\renewcommand{\\headrulewidth}{0pt}\n',
 '\\renewcommand{\\footrulewidth}{0pt}\n',
 '\n',
 '\\addtolength{\\oddsidemargin}{-0.5in}\n',
 '\\addtolength{\\evensidemargin}{-0.5in}\n',
 '\\addtolength{\\textwidth}{1in}\n',
 '\\addtolength{\\topmargin}{-.5in}\n',
 '\\addtolength{\\textheight}{1.0in}\n',
 '\n',
 '\\urlstyle{same}\n',
 '\\raggedbottom\n',
 '\\raggedright\n',
 '\\setlength{\\tabcolse